In [1]:
import spacy 
import torch
import json
from nlp_utils import *
import random

# Prep

In [2]:
with open ("data.json", 'r') as f:
    intents = json.load(f)["intents"]
    
data = torch.load("model.save")

model_state = data["model_state"]

input_size = data["input_size"]
hidden_size = data["hidden_size"]
num_classes = data["num_classes"]

tag_translator = data["tag_translator"]

device = torch.device("cuda")
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
model.load_state_dict(model_state)
model.eval()
nlp = spacy.load("en_core_web_lg")


# Bot

In [3]:
print("Type 'quit' to quit.")

user_input = ""

tag = ""


# while user_input != "quit":
while tag != "GoodBye":
    user_input = input()
    print("\tYou:",user_input)
    
    user_input_vector = torch.from_numpy(tokenize_to_vec(user_input, nlp))
    user_input_vector = user_input_vector.reshape(1, user_input_vector.shape[0])
    user_input_vector = user_input_vector.to(device)
    
    output = model(user_input_vector)
    _, prediction = torch.max(output, dim=1)

    probs = torch.softmax(output, dim=1)
    prob = probs[0][prediction.item()]


    if prob >= 0.75: 
        # get tag
        for key in tag_translator:
            if tag_translator[key] == prediction.item():
                tag = key
        # get response
        for intent in intents:
            if intent["intent"] == tag:
                print("\tChatBot:", random.choice(intent["responses"]))
                print()
                print("prob:", prob.item(), "| tag:", tag)
                print()
                print()
    else:
        print("prob:", prob.item())
        print("Chatbot: I don't understand.")
        print()
        print()
            
        
    

    

Type 'quit' to quit.
	You: hello there 
	ChatBot: Hi human, please tell me your GeniSys user

prob: 1.0 | tag: Greeting


	You: my name is alice 
	ChatBot: You are <HUMAN>! How can I help?

prob: 0.914756178855896 | tag: CurrentHumanQuery


	You: who am i 
	ChatBot: Please look at the camera

prob: 0.9991425275802612 | tag: WhoAmI


	You: how are you 
	ChatBot: Hello, I am good thank you, how are you? Please tell me your GeniSys user

prob: 0.9999338388442993 | tag: CourtesyGreeting


	You: are you good
	ChatBot: Hi, how are you? I am great thanks! Please tell me your GeniSys user

prob: 0.7716938853263855 | tag: CourtesyGreeting


	You: what do you call me 
	ChatBot: You are <HUMAN>! How can I help?

prob: 0.9907564520835876 | tag: CurrentHumanQuery


	You: alice
	ChatBot: I’m sorry, I’m afraid I can’t do that!

prob: 0.9101258516311646 | tag: PodBayDoor


	You: door
	ChatBot: I’m sorry, I’m afraid I can’t do that!

prob: 1.0 | tag: PodBayDoor


	You: tell me a joke
	ChatBot: Two cows